<a href="https://colab.research.google.com/github/et-tu-atulu/dog_breed_identification/blob/master/VG19_pup_breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#extract the zip file here
import os
from zipfile import ZipFile
zipobj=ZipFile(os.path.abspath('.')+'/drive/My Drive/dataset.zip', 'r')
zipobj.extractall()

In [0]:
!pip install tf-nightly-gpu

     |████████████████████████████████| 396.1MB 47kB/s 
     |████████████████████████████████| 61kB 23.2MB/s 
     |████████████████████████████████| 501kB 49.5MB/s 
     |████████████████████████████████| 3.2MB 49.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/51/3e/a3/b351fae0cbf15373c2136a54a70f43fea5fe91d8168a5faaa4
Successfully built opt-einsum


In [0]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D



image_gen_train = ImageDataGenerator(rescale = 1./255, horizontal_flip = True, zoom_range = 0.2, rotation_range=90)

train_gen = image_gen_train.flow_from_directory(batch_size=32, directory = os.path.abspath('.')+'/train', shuffle = True, target_size =(150, 150), class_mode = 'sparse')

image_gen_test = ImageDataGenerator(rescale = 1./255)

test_gen = image_gen_test.flow_from_directory(batch_size=32, directory=os.path.abspath('.')+'/test', target_size=(150, 150), class_mode='sparse')



Found 16418 images belonging to 120 classes.
Found 4162 images belonging to 120 classes.


# New Section

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras import layers
import tensorflow_hub as hub
import numpy as np


import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

#

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

x = base_model.output
x = Flatten()(x)
predictions = Dense(120, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

    
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [0]:


EPOCHS = 50
history = model.fit_generator(train_gen,
                    steps_per_epoch=513,         
                    epochs=EPOCHS,
                    validation_data=test_gen,
                    validation_steps=130)

Epoch 1/50
513/513 [==============================] - 162s 317ms/step - loss: 4.8175 - acc: 0.0634 - val_loss: 4.1794 - val_acc: 0.1310
Epoch 2/50
513/513 [==============================] - 144s 281ms/step - loss: 4.2075 - acc: 0.1299 - val_loss: 4.1316 - val_acc: 0.1494
Epoch 3/50
513/513 [==============================] - 143s 278ms/step - loss: 3.9767 - acc: 0.1589 - val_loss: 4.0059 - val_acc: 0.1765
Epoch 4/50
513/513 [==============================] - 142s 277ms/step - loss: 3.8166 - acc: 0.1851 - val_loss: 4.1139 - val_acc: 0.1707
Epoch 5/50
513/513 [==============================] - 141s 274ms/step - loss: 3.7069 - acc: 0.2015 - val_loss: 4.0568 - val_acc: 0.1990
Epoch 6/50
513/513 [==============================] - 142s 276ms/step - loss: 3.5921 - acc: 0.2149 - val_loss: 4.2478 - val_acc: 0.1714
Epoch 7/50
513/513 [==============================] - 142s 277ms/step - loss: 3.5185 - acc: 0.2343 - val_loss: 4.1938 - val_acc: 0.1857
Epoch 8/50
513/513 [============================